# Capstone Project

This is the coursera capstone project.

In [256]:
#Import necessary modules
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests #Makes web requests

# 1._ Scrap Wikipedia

In [257]:
#Get the source
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#html format
soup=BeautifulSoup(source,'lxml')

#Get the table
table=soup.find('table', class_='wikitable')

#print(table.prettify())

In [258]:
#We iterate to get a list for each column values
data_list=[[],[],[]]
for tr in table.find_all('tr'): #Goes through all the <tr> tags
    i=0
    for td in tr.find_all('td'):
        try:
            data_list[i].append(td.text.rstrip())#rstrip to remove the final \n
        except:
            data_list[i].append("Error")
        finally:
            i=i+1

In [259]:
#I have all the data in a list with three lists
#I separate the data

Postal_code_list=data_list[0][:]
Borough_list=data_list[1][:]
Neighborhood_list=data_list[2][:]

#Create a dictionary and create a frame with it
dictionary = {'Postal_code':Postal_code_list,
              'Borough':Borough_list,
              'Neighborhood':Neighborhood_list}
df = pd.DataFrame(dictionary, columns = ['Postal_code','Borough','Neighborhood'])

# 2._ Clean the data

We have all the information from the wikipedia table stored in the dataframe.

Now we have to clean the data according to:
 - Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.
 - In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [260]:
df.shape

(180, 3)

 - Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [261]:
#Get rid of all the rows with "Not assigned" in Borough
df=df[~((df['Borough'] == 'Not assigned'))]
df.shape

(103, 3)

 - If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [262]:
#We use boolean indexing
df.loc[df['Neighborhood'] == 'Not assigned', 'Neighborhood'] = df['Borough']
df.shape

(103, 3)

 - More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Actually now the table in the wikipedia page has this job done already.
##### Check
We can make sure that our table satisfies that, and also make sure that all the rows with the same postal code also have the same Borough.

For that, we check the dimension of the dataframe as it is, then grouped by Postal_code and Borough, and grouped by Postal_code. We see that they all have the same lenght.

In [263]:
g_pc=df.groupby(['Postal_code'])
g_pc_b = df.groupby(['Postal_code','Borough'])
print(df.shape)
print(len(g_pc))
print(len(g_pc_b))


#df=df.groupby(['Postal_code','Borough'])['Neighborhood'].apply(','.join)
#df

(103, 3)
103
103


In [265]:
#To make it look like in the exercise, we can change the "/" for a ",".
df.reset_index(level=None, drop=True, inplace=True)
df.replace("/", ",", inplace=True, regex=True)

# 3._ Result

In [266]:
df

,Postal_code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park , Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway , Montgomery Road , Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing CentrE
101,M8Y,Etobicoke,"Old Mill South , King's Mill Park , Sunnylea ,..."


# We end up with 103 rows